In [11]:
import pandas as pd
import geopandas as gpd
import folium

In [12]:
# Load data
stations = gpd.read_file('/Users/vinaymadhugiri/Downloads/NJTransit_Rail_Stations_-3939820597305918471.geojson')  # NJ Transit stations GeoJSON
lines = gpd.read_file('/Users/vinaymadhugiri/Downloads/Rail_Lines_of_NJ_Transit.geojson')   # Rail lines GeoJSON
municipalities = gpd.read_file('/Users/vinaymadhugiri/Downloads/NJ_Municipal_Boundaries_3424_930771950455602469.geojson')  # Municipalities GeoJSON

In [5]:
# Create a map
nj_map = folium.Map(location=[40.0583, -74.4057], zoom_start=8)

In [6]:
# Add rail lines
folium.GeoJson(lines, name="Rail Lines").add_to(nj_map)

In [14]:
# Join the Stations and Lines
stations_with_lines = stations.merge(lines, on='LINE_CODE', how='inner')

In [13]:
# Perform spatial joins with unique suffixes

stations_with_data = gpd.sjoin(
    stations_with_lines, municipalities, how="left", predicate="within", lsuffix="stations_lines", rsuffix="municipalities"
)


ValueError: 'left_df' should be GeoDataFrame, got <class 'pandas.core.frame.DataFrame'>

In [8]:
# Add stations to the map
for idx, row in stations_with_data.iterrows():
    popup_text = f"""
    <b>Station Name:</b> {row['STATION_ID']}<br>
    <b>Rail Line:</b> {row['LINE_NAME']}<br>
    <b>Municipality:</b> {row['MUN_LABEL']}<br>
    <b>Population:</b> {row['POP2020']:,}
    """
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(nj_map)


In [9]:
# Save the map
nj_map.save("nj_transit_map.html")

In [10]:
# Spatial join
municipalities_without_stations = municipalities[~municipalities.intersects(stations.unary_union)]

# Add population and sort
municipalities_without_stations = municipalities_without_stations[['NAME', 'POP2020']].sort_values(by='POP2020', ascending=False)
print(municipalities_without_stations.head(25))

                               NAME  POP2020
442                     Jersey City   292449
141               Lakewood Township   135158
132             Toms River Township    95438
147                  Brick Township    73620
562                          Camden    71791
282                         Bayonne    71686
303                      Union City    68589
237               Franklin Township    68364
222             Old Bridge Township    66876
541             Gloucester Township    66034
315           North Bergen Township    63361
440              Irvington Township    61176
247             Piscataway Township    60804
39                         Vineland    60780
155                Jackson Township    58544
519  Parsippany-Troy Hills Township    56162
170                 Howell Township    53537
308                   West New York    52912
226         East Brunswick Township    49715
316            West Orange Township    48843
540             Washington Township    48677
517       